In [1]:
## 
using Revise
includet("../src/AeroMDAO.jl")
includet("../src/MathTools.jl")

┌ Info: Precompiling Revise [295af30f-e4ad-537b-8983-00126c2a3abe]
└ @ Base loading.jl:1278
┌ Info: Precompiling StaticArrays [90137ffa-7385-5640-81b9-e52037218182]
└ @ Base loading.jl:1278
┌ Info: Precompiling Interpolations [a98d9a8b-a2ab-59e6-89dd-64a1c18fca59]
└ @ Base loading.jl:1278
┌ Info: Precompiling Rotations [6038ab10-8711-5258-84ad-4b1120ba62dc]
└ @ Base loading.jl:1278


In [2]:
##
using .AeroMDAO
using .MathTools: linspace, tuparray
using DelimitedFiles
using Rotations

In [12]:
## Wing section setup
foilpath = "../airfoil_database/ys930.dat"

num_secs = 3

coords = read_foil(foilpath)
foils = [ coords for i in 1:num_secs ]
airfoils = Foil.(foils)

wing_chords = [1, 0.5, 0.2]
wing_twists = [5, 2, 0]
wing_spans = [2, 0.2]
wing_dihedrals = [0, 45]
wing_sweeps = [15, 60]

wing_right = HalfWing(airfoils, wing_chords, wing_spans, wing_dihedrals, wing_sweeps, wing_twists)
wing = Wing(wing_right, wing_right)
print_info(wing_right)

Span: 2.137669413923499 m
Area: 1.4709312378592305 m²
MAC: 0.759660297239915 m
Aspect Ratio: 3.106624161354137


Assembly

In [13]:
wing_panels = make_panels(wing_right, spanwise_panels = 5, chordwise_panels = 1)
test_panels = mesh_wing(wing, spanwise_panels = 5, chordwise_panels = 15)

560-element Array{Panel3D,1}:
 Panel3D([0.9961946980917455, 0.0, -0.08715574274765817], [0.983972943969317, 0.0, -0.08301749690023098], [0.9930549564811364, -0.4, -0.06955465345679959], [1.004074518147755, -0.4, -0.07321454386837663])
 Panel3D([0.983972943969317, 0.0, -0.08301749690023098], [0.9480102251689929, 0.0, -0.06978506366892208], [0.9606241849218112, -0.4, -0.057835491335358286], [0.9930549564811364, -0.4, -0.06955465345679959])
 Panel3D([0.9480102251689929, 0.0, -0.06978506366892208], [0.8897942320828084, 0.0, -0.05172964558567597], [0.9081432746866951, -0.4, -0.041892768459414066], [0.9606241849218112, -0.4, -0.057835491335358286])
 Panel3D([0.8897942320828084, 0.0, -0.05172964558567597], [0.8123233251653206, 0.0, -0.028851728355853833], [0.8383103418119953, -0.4, -0.021711229691072746], [0.9081432746866951, -0.4, -0.041892768459414066])
 Panel3D([0.8123233251653206, 0.0, -0.028851728355853833], [0.719354719971447, 0.0, -0.003755828898200489], [0.7545199963721532, -0.4, 0.00

Panel case

In [14]:
ρ = 1.225
uniform = Uniform(10.0, 5.0, 0.0)
@time lift, drag = solve_case(wing_panels, uniform, ρ)

V = uniform.mag
S = projected_area(wing_right)
cl = lift/(ρ/2 * V^2 * S)
cdi = drag/(ρ/2 * V^2 * S)
println("Lift: $(sum(lift)), Drag: $(sum(drag))")
println("Lift Coefficient: $(sum(cl)), Drag Coefficient: $(sum(cdi))")
println("Lift-to-Drag Ratio (L/D): $(sum(cl)/sum(cdi))")

  0.057635 seconds (197.81 k allocations: 10.247 MiB)
Lift: 7.32395489386421, Drag: 1.0858771743343891
Lift Coefficient: 0.0812918855074436, Drag Coefficient: 0.012052641531843676
Lift-to-Drag Ratio (L/D): 6.7447360226110105


In [15]:
using Plots, LaTeXStrings

In [16]:
plotlyjs()

Plots.PlotlyJSBackend()

Wing Plot

In [17]:
# wing_plot = plot_setup(coordinates(wing))
# wing_sects = plot_setup.(sections(wing))
# wing_pts = horseshoe_vortex.(wing_panels)
wing_collocs = horseshoe_collocation.(wing_panels)
pan_coords = (tuparray ∘ panel_coords).(test_panels)
spans = [ pt[2] for pt in wing_collocs ];

In [18]:
##
plot(spans, cl)

<!DOCTYPE html>
 
 
 Plots.jl

In [19]:
##
plot(spans, cdi)

<!DOCTYPE html>
 
 
 Plots.jl

In [20]:
##
plot(xaxis = "x", yaxis = "y", zaxis = "z", aspect_ratio = :equal, zlim = (-0.5, 5.0))
plot!.(pan_coords, color = :black,label = :none)

scatter!(wing_collocs, c = :grey, markersize = 1, label = "Wing Collocation Points")

# plot!.(wing_pts, c = :black, label = :none)
# wing_lines = [ horseshoe_lines(panel, uniform) for panel in wing_panels ]

# lines = [ [ tuparray([line.r1'; line.r2']) for line in horseshoe.vortex_lines ] for horseshoe in wing_lines ]
# [ [ plot!(line, c = :darkblue, label = :none) for line in horseshoe ] for horseshoe in lines ]

<!DOCTYPE html>
 
 
 Plots.jl